# 텍스트 분석
***

사이킷런의 내장 데이터인 20 뉴스그룹 데이터를 이용해 텍스트 분류를 적용.

In [1]:
from sklearn.datasets import fetch_20newsgroups

불필요한 정보가 있는 headers, footers, quotes를 제거한다.

In [11]:
train = fetch_20newsgroups(subset = 'train', random_state = 42, remove = ('headers', 'footers', 'quotes'))

In [13]:
test = fetch_20newsgroups(subset = 'test', random_state = 42, remove = ('headers', 'footers', 'quotes'))

In [12]:
train.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [14]:
test.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [7]:
import pandas as pd
import numpy as np

In [16]:
X = train['data']
target = test['data']

In [22]:
y = train['target']
actual_target = test['target']

학습 데이터와 예측 데이터를 분할한다.

In [19]:
len(X), len(target)

(11314, 7532)

총 2가지 벡터 변환을 사용한다.
- 1. CountVectorizer
- 2. TfidfVectorizer

In [20]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [33]:
count_vect = CountVectorizer(max_features = 5000)

전체 단어 중 빈도 기준 상위 5000개 단어만 사용한다.

In [34]:
X_count_vect = count_vect.fit_transform(X)

In [35]:
X_count_vect.shape

(11314, 5000)

In [36]:
target_count_vect = count_vect.transform(target)

In [44]:
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

In [37]:
lr_clf = LogisticRegression()

In [45]:
lr_clf.fit(X_count_vect, y)

LogisticRegression()

In [39]:
pred = lr_clf.predict(target_count_vect)

In [40]:
from sklearn.metrics import accuracy_score

In [41]:
acc = accuracy_score(actual_target, pred)

In [43]:
print(f'Count기반의 벡터화 적용 모델의 정확도는 {acc}')

Count기반의 벡터화 적용 모델의 정확도는 0.580323951141795


TfidfVectorizer 적용

In [52]:
tfidf_vect = TfidfVectorizer(max_features = 5000, stop_words = 'english')

In [53]:
X_tfidf_vect = tfidf_vect.fit_transform(X)

In [54]:
target_tfidf_vect = tfidf_vect.transform(target)

In [55]:
lr_clf.fit(X_tfidf_vect, y)

LogisticRegression()

In [56]:
pred = lr_clf.predict(target_tfidf_vect)

In [57]:
acc = accuracy_score(actual_target, pred)

In [58]:
print(f'Tfidf기반의 벡터화 적용 모델의 정확도는 {acc}')

Tfidf기반의 벡터화 적용 모델의 정확도는 0.6471056824216675


Tfidf 기반의 벡터화 적용 모델이 정확도가 6% 높은 것으로 나타났다.
***
### 추가 실험 - Pipeline + GridSearchCV이용

- ngram_range
- max_df

In [59]:
from sklearn.pipeline import Pipeline

In [75]:
pl = Pipeline([('tfidf_vec', TfidfVectorizer(stop_words = 'english', max_features = 5000, lowercase = True)),
              ('lr_clf', LogisticRegression(random_state = 42))])

In [76]:
param = {'tfidf_vec__ngram_range' : [(1, 1), (1, 2), (1, 3)],
        'tfidf_vec__max_df' : [100, 300, 500, 700],
        'lr_clf__C' : [1, 5, 10]}

In [77]:
from sklearn.model_selection import GridSearchCV

In [78]:
cv = GridSearchCV(pl, param_grid = param, cv = 5, scoring = 'accuracy', verbose = 0)

In [79]:
cv.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf_vec',
                                        TfidfVectorizer(max_features=5000,
                                                        stop_words='english')),
                                       ('lr_clf',
                                        LogisticRegression(random_state=42))]),
             param_grid={'lr_clf__C': [1, 5, 10],
                         'tfidf_vec__max_df': [100, 300, 500, 700],
                         'tfidf_vec__ngram_range': [(1, 1), (1, 2), (1, 3)]},
             scoring='accuracy')

In [80]:
pred = cv.predict(target)

In [82]:
acc = accuracy_score(actual_target, pred)

In [83]:
print(f'Pipeline + GridSearchCV 모델의 정확도는 {acc}')

Pipeline + GridSearchCV 모델의 정확도는 0.6469729155602761
